<a href="https://colab.research.google.com/github/HenrikSandberg/BrickBreaker/blob/master/DrivieOS_CoLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/rslim087a/track

fatal: destination path 'track' already exists and is not an empty directory.


In [5]:

import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import cv2
import pandas as pd
import ntpath

# TensorFlow related imports
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(tensorflow.__version__)
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'



1.15.0


In [8]:
np.random.seed(0)
datadir = 'track'
colums = ['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed']
data = pd.read_csv(os.path.join(datadir, 'driving_log.csv'), names=colums)
# pd.set_option('display.max_colwidth', None)
data.head()

,center,left,right,steering,throttle,reverse,speed
0,C:\Users\Amer\Desktop\new_track\IMG\center_201...,C:\Users\Amer\Desktop\new_track\IMG\left_2018_...,C:\Users\Amer\Desktop\new_track\IMG\right_2018...,0.0,0.0,0.0,0.649786
1,C:\Users\Amer\Desktop\new_track\IMG\center_201...,C:\Users\Amer\Desktop\new_track\IMG\left_2018_...,C:\Users\Amer\Desktop\new_track\IMG\right_2018...,0.0,0.0,0.0,0.627942
2,C:\Users\Amer\Desktop\new_track\IMG\center_201...,C:\Users\Amer\Desktop\new_track\IMG\left_2018_...,C:\Users\Amer\Desktop\new_track\IMG\right_2018...,0.0,0.0,0.0,0.622910
3,C:\Users\Amer\Desktop\new_track\IMG\center_201...,C:\Users\Amer\Desktop\new_track\IMG\left_2018_...,C:\Users\Amer\Desktop\new_track\IMG\right_2018...,0.0,0.0,0.0,0.619162
4,C:\Users\Amer\Desktop\new_track\IMG\center_201...,C:\Users\Amer\Desktop\new_track\IMG\left_2018_...,C:\Users\Amer\Desktop\new_track\IMG\right_2018...,0.0,0.0,0.0,0.615438


In [0]:
def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail


def load_img_steering(datadir, df):
    image_path = []
    steering = []
    for i in range(len(data)):
        indexed_data = data.iloc[i]
        center, left, right = indexed_data[0], indexed_data[1], indexed_data[2]
        image_path.append(os.path.join(datadir, center.strip()))
        steering.append(float(indexed_data[3]))
    image_path = np.asarray(image_path)
    steering = np.asarray(steering)
    return (image_path, steering)


def img_preprocess(img):
    img = mpimg.imread(img)
    img = img[60:135, :, :]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = cv2.GaussianBlur(img, (3, 3), 0)
    img = cv2.resize(img, (200, 66))
    return img / 255


def nvidia_model():
    _model = Sequential([
        # Convolution layers
        Conv2D(24, kernel_size=(5, 5), strides=(2, 2), input_shape=(66, 200, 3), activation='elu'),
        Conv2D(36, kernel_size=(5, 5), strides=(2, 2), activation='elu'),
        Conv2D(48, kernel_size=(5, 5), strides=(2, 2), activation='elu'),
        Conv2D(64, kernel_size=(3, 3), activation='elu'),
        Conv2D(64, kernel_size=(3, 3), activation='elu'),

        # Fully connected layers
        Flatten(),
        Dense(100, activation='elu'),
        Dense(50, activation='elu'),
        Dense(10, activation='elu'),
        Dense(1)
    ])
    # Compile with optimizer
    adam = Adam(learning_rate=0.006)
    _model.compile(loss='mse', optimizer=adam)
    return _model


def print_history(_history):
    plt.plot(_history.history['loss'])
    plt.plot(_history.history['val_loss'])
    plt.legend(['Training', 'Validation'])
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.show()

In [10]:

# Handle data
data['center'] = data['center'].apply(path_leaf)
data['left'] = data['left'].apply(path_leaf)
data['right'] = data['right'].apply(path_leaf)
data.head()

num_bins = 25
samples_per_bin = 300
hist, bins = np.histogram(data['steering'], num_bins)
center = (bins[:-1] + bins[1:]) * 0.5

# Drop data so it is more even
remove_list = []
for j in range(num_bins):
    list_ = []
    for i in range(len(data['steering'])):
        if bins[j] <= data['steering'][i] <= bins[j + 1]:
            list_.append(i)
    list_ = shuffle(list_)
    list_ = list_[samples_per_bin:]
    remove_list.extend(list_)

print('removed: ', len(remove_list))
data.drop(data.index[remove_list], inplace=True)
print('removing: ', len(data))

# Load images
(image_path, steering) = load_img_steering(datadir + '/IMG', data)

# Train data
X_train, X_val, y_train, y_val = train_test_split(image_path, steering, test_size=0.2, random_state=6)

print('Trainging Samples: {}\nValidation Sample: {}'.format(len(X_train), len(X_val)))

# Generate Validataion and Training data in np.arrays
X_train = np.array(list(map(img_preprocess, X_train)))
X_val = np.array(list(map(img_preprocess, X_val)))

# Generate noise in data
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    shear_range=0.1,
    rotation_range=10)

datagen.fit(X_train)
# batches = datagen.flow(X_train, y_train, batch_size=300)
# X_batch, y_batch = next(batches)

# Train the data
model = nvidia_model()

# Load the pre-trained model if it exist
try:
    model.load_weights('model.h5')
except Exception as e:
    print('Exception:\n{}'.format(str(e)))

batch_size = 100

removed:  2690
removing:  1363
Trainging Samples: 1090
Validation Sample: 273
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Exception:
Unable to open file (unable to open file: name = 'model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


In [0]:
history = model.fit_generator(
    datagen.flow(X_train, y_train, batch_size=batch_size),
    steps_per_epoch=batch_size*3,
    epochs=10,
    validation_data=(X_val, y_val),
    validation_steps=len(X_val),
    verbose=1,
    shuffle=1)

Epoch 1/10
146/300 [=============>................] - ETA: 2:27 - loss: 12.7124

In [0]:
print_history(history)
model.save('model.h5')